<p>
<font size='5' face='Georgia, Arial'>IIC2233 Apunte Programación Avanzada</font><br>
<font size='1'>Basado en: &copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. Modificado en 2018-1, 2019-2, 2020-1 y 2020-2 por Equipo IIC2233</font>
</p>

# Tabla de contenidos
1. [Polimorfismo](#Polimorfismo)
    1. [Overriding](#Overriding)
    2. [Overloading](#Overloading)
        1. [Overloading de operadores en Python](#Overloading-de-operadores-en-Python)
    3. [`__repr__` vs `__str__`](#__repr__-vs-__str__)
2. [*Duck typing*](#Duck-typing)

# Polimorfismo

El **polimorfismo** se refiere a "la propiedad por la que es posible enviar mensajes sintácticamente iguales a objetos de tipos distintos" ([Wikipedia](https://es.wikipedia.org/wiki/Polimorfismo_%28inform%C3%A1tica%29), 2017)). Básicamente se trata de utilizar objetos de distinto tipo con la misma *interfaz*. Dos mecanismo para proveer polimorfismo son _overriding_ y _overloading_.

- ***Overriding***: ocurre cuando se implementa un método en una subclase que sobreescribe la implementación del mismo método en la super clase
   
- ***Overloading***: es la capacidad de definir un método con el mismo nombre pero con distinto número y tipo de argumentos. Es la capacidad de una función de ejecutar distintas acciones dependiendo del tipo y número de argumentos que recibe. 
  
Python no soporta _function overloading_. No se puede definir la función más de una vez con distintos tipos y números de argumentos y esperar que ambas definiciones sean consideradas por el programa. Sin embargo, se puede "simular" usando algunos parámetros con valores por defecto o número de argumentos variables.
   

## *Overriding*
Como se mencionó anteriormente, una subclase puede sobreescribir la implementación de los distintos métodos que hereda. A continuación se encuentra un ejemplo en el que se crea una clase superior de nombre `Variable`, la cual almacena un conjunto de datos en el atributo `data`. Se definen tres subclases: `Ingresos`, `Comuna` y `Puesto`. Cada uno, como subclase, posee un atributo `data`, y una implementación distinta del método `representante`. Este método se usa para obtener un valor a partir del conjunto de datos. Algunos ejemplos de valores "representantes" pueden ser el promedio, la mediana, o la moda.

El siguiente diagrama representa este diseño:

![](img/OOP_polimorfismo.png)

Se define entonces cómo debe funcionar el método `representante` para cada subclase.

- Si los datos son de tipo `Ingresos`, el valor representante es el promedio.
- Si los datos son de tipo `Comuna`, el valor representante es la comuna que más se repite. 
- Si los datos son del tipo `Puesto` de trabajo, entonces el valor representante es el que tiene el puesto más alto según la jerarquía especificada en una lista de categorías.

La implementación se ve de la siguiente manera:

In [1]:
import statistics

class Variable:
    def __init__(self, data):
        self.data = data

    def representante(self):
        pass


class Ingresos(Variable):
    def representante(self):
        return statistics.mean(self.data)


class Comunas(Variable):
    def representante(self):
        return statistics.mode(self.data)


class Puestos(Variable):
    # Ordenadas de menor a mayor
    # Este es un atributo de la clase Puestos, compartida por todas sus instancias
    # Este tipo de atributo se accede con la notación NombreDeLaClase.atributoClase
    # Por ejemplo: Puestos.categorias
    categorias = ['Alumno en Practica', 'Analista', 'SubGerente', 'Gerente']

    def representante(self):
        # Paso 1: Transformar la lista en lista de números, donde 0 es alumno en práctica y 3 gerente
        puestos = []
        for cargo in self.data:
            puestos.append(Puestos.categorias.index(cargo))
        # Paso 2: Vemos cuál es el máximo
        maximo = max(puestos)
        # Paso 3: Retornar cargo asociado
        return Puestos.categorias[maximo]

Vemos que cada subclase define su propia implementación del método `representante`. Cuando se invoca a un método sobre un tipo de datos, primero se busca el método en la definición del tipo de datos correspondiente. Por ejemplo, si estamos en un objeto de tipo `Comunas`, se invoca el método `representante` definido en la clase `Comunas`. Si no se llegara a encontrar el método en la definición de una clase, entonces se busca si está implementado en la clase superior.

In [2]:
lista_pesos = Ingresos([50, 80, 90, 150, 45, 65, 78, 89, 59, 77, 90])
lista_comunas = Comunas(['Providencia', 'Macul' , 'La Reina' ,'Santiago', 'Providencia', 'Puente Alto',
                        'Macul', 'Santiago', 'Santiago'])
lista_puestos = Puestos(['SubGerente', 'Analista','SubGerente','Analista','Alumno en Practica',
                        'Alumno en Practica'])

print(lista_pesos.representante())
print(lista_comunas.representante())
print(lista_puestos.representante())

79.36363636363636
Santiago
SubGerente


Podemos ver que, si bien las clases `Ingresos`, `Comunas` y `Puestos` heredan de la misma clase, a pesar de tener distintos tipos de datos, para cada una de ellas podemos llamar de la misma manera a su método `representante` y, de acuerdo a la clase que corresponda, se llama a la versión correcta del método.

Esto es un ejemplo de polimorfismo: se invoca el mismo método sobre objetos de distinto tipo, y cada uno lo interpreta de acuerdo a su propia definición.

## *Overloading*

A diferencia de otros lenguajes, como C++ o Java, python no soporta _function overloading_, es decir, no es posible definir dos veces la misma función con diferente tipo o número de argumentos, es decir, el siguiente código no se podrá ejecutar:

In [3]:
def funcion(arg):
    print(arg)


def funcion(arg1, arg2):
    print(arg1, arg2)
    
funcion('este')
funcion('codigo', 'fallará')

TypeError: funcion() missing 1 required positional argument: 'arg2'

Profundizando un poco más, al leer el error entregado se hace evidente que la definición que se está considerando es la segunda, pues la ejecución de `funcion('este')` falla al dar un solo argumento en lugar de dos, es decir, python está considerando solamente la definición más nueva.

A pesar de lo anterior, python sí permite un tipo de _overloading_, el _overloading_ de sus operadores *built-in*.

### *Overloading* de operadores en Python
Existen muchos operadores en Python que funcionan para varias de las clases *built-in*. Por ejemplo, el operador "+" puede sumar dos números, concatenar dos strings, mezclar dos listas, etc. (dependiendo de la clase con la que estemos trabajando). Esto es un ejemplo de `overloading`; el mismo operador funciona de distinta manera de acuerdo al tipo de los argumentos que recibe.

In [4]:
a = [1, 2, 3, 4]
b = [5, 6, 7, 8]
print(a + b)
c = "Hola"
d = " Mundo"
print(c + d)

[1, 2, 3, 4, 5, 6, 7, 8]
Hola Mundo


Además, Python nos permite personalizar el método `__add__` para que el operador "+" funcione en algún tipo de clase específica que necesitemos. Por ejemplo, supongamos una clase que representa un carro de compra:

In [5]:
class Carro:

    def __init__(self, pan, leche, agua):
        self.pan = pan
        self.leche = leche
        self.agua = agua
    
    def __add__(self, otro):
        
        suma_pan = self.pan + otro.pan
        suma_leche = self.leche + otro.leche
        suma_agua = self.agua + otro.agua
            
        return Carro(suma_pan, suma_leche, suma_agua)
    
    def __str__(self):
        return f"Pan:{self.pan}, Leche:{self.leche}, Agua:{self.agua}"

In [6]:
carro_1 = Carro(1, 2, 3)
carro_2 = Carro(3, 4, 5)
carro_sumado = carro_1 + carro_2
print(carro_sumado)

Pan:4, Leche:6, Agua:8


De la misma forma, podemos personalizar la mayoría de los operadores. Por ejemplo, para personalizar el operador "menor que" implementamos `__lt__` (del inglés *less than*):

In [7]:
import math

class Vector: 
    """Vector desde el origen"""
    def __init__(self, x, y): 
        self.x = x 
        self.y = y
        
    @property
    def magnitud(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)
    
    def __lt__(self, otro_punto):
        return self.magnitud < otro_punto.magnitud

v1 = Vector(2,4)
v2 = Vector(8,3)
print(v1 < v2)

True


**Para poner en práctica el *overriding* de operadores puedes realizar los ejercicios propuestos 1.2 y 1.3.**

## `__repr__` vs `__str__`

Podemos implementar los métodos `__repr__` y `__str__` para entregar una representación en texto de nuestro objeto. Estos métodos deben retornar un *string*, el que podrá ser usado por la función `print`. Si se implementan ambos, `print` utiliza `__str__`.

La diferencia entre  `__str__` y `__repr__` es sutil. Si bien ambos devuelven una representación del objeto en forma de *string*, cada representación persigue un objetivo distinto. Por una parte, `__str__` busca devolver una representación legible (*human-readable*) del objeto. Es como si un usuario del programa quisiera leer esa información, y por eso se usa para `print`. Por otra parte, `__repr__` tiene por objetivo ofrecer una representación completa y sin ambigüedades del objeto. Es como si un desarrollador del programa quisiera leer esa información.

El siguiente ejemplo define una clase `Fraccion`, con una implementación para `__repr__` y una para `__str__`.

In [8]:
class Fraccion:
    def __init__(self, numerador, denominador): 
        self.numerador = numerador 
        self.denominador = denominador
        
    def __repr__(self):
        return f"Fraccion({self.numerador}, {self.denominador})"
    
    def __str__(self):
        return f"{self.numerador} / {self.denominador}"
    
frac = Fraccion(3, 4)

In [9]:
repr(frac)

'Fraccion(3, 4)'

In [10]:
str(frac)

'3 / 4'

In [11]:
print(frac)

3 / 4


Si no implementamos el método `__str__`, `print` va a imprimir el *string* que retorna la función `__repr__`

In [12]:
class Fraccion: 
    def __init__(self, numerador, denominador): 
        self.numerador = numerador 
        self.denominador = denominador
        
    def __repr__(self):
        return f"Fraccion({self.numerador}, {self.denominador})"
    
frac = Fraccion(3, 4)
print(frac)

Fraccion(3, 4)


**Revisa el ejercicio 1.1 para notar otra diferencia entre `__str__` y `__repr__`.**

# *Duck typing*

> "If it walks like a duck and quacks like a duck then it is a duck" 
(no importa de qué tipo sea un objeto mientras contenga la acción)
 
*Duck typing* es una característica de algunos lenguajes que hace que el polimorfismo sea menos atractivo, ya que el lenguaje por sí sólo es capaz de generar comportamiento polimórfico sin la necesidad de implementar el polimorfismo a través de la herencia. 

Consideremos las siguientes clases:

In [13]:
class Pato:
    def gritar(self):
        print("Quack!")
        
    def caminar(self):
        print("Caminando como un pato")        
    
class Persona:
    def gritar(self):
        print("¡Ahhh!")
        
    def caminar(self):
        print("Caminando como un humano")

El diagrama que los representa es el siguiente:

![](img/OOP_ducktyping.png)

Es claro que si creamos un objeto de tipo `Pato` y un objeto de tipo `Persona`, se llamará a los métodos `gritar` y `caminar` que correspondan a la clase.

In [14]:
donald = Pato()
enzo = Persona()
donald.gritar()
enzo.gritar()

Quack!
¡Ahhh!


Pero si escribimos una función que recibe un argumento, no sabemos, al momento de programarlo, qué tipo de dato recibirá este objeto. Y no necesitamos saberlo, pues el mecanismo de *duck typing* determinará al momento de ejecutar, qué método se invocará, de acuerdo con el tipo de dato.

In [15]:
def activar(pato):  # Esto, en otro tipo de lenguaje, obligaría a que pato sea del tipo "Pato", por lo tanto
    pato.gritar()   # la función activar no podría ser llamada con un argumento tipo "Persona"
    pato.caminar()

donald = Pato()
juan = Persona()
activar(donald)
activar(juan)

Quack!
Caminando como un pato
¡Ahhh!
Caminando como un humano


En este ejemplo hay dos clases distintas, `Pato` y `Persona`, sin ninguna relación de herencia entre ellas. Cada una tiene implementados los métodos `gritar` y `caminar`. La función `activar` recibe un argumento de nombre `pato`, pero no sabe (ni le interesa) si es un objeto de tipo `Pato` o `Persona`; simplemente llama a los métodos `gritar` y `caminar`, y en ese momento se determina si la clase a la cual pertenece el argumento `Pato` contiene una implementación del método que se necesita.

Este comportamiento puede parecer obvio, sobre todo para quienes solo han programado en Python, sin embargo en otros lenguajes de programación, como C, C++, Java ó C#, se obliga a que los argumentos tengan un tipo de dato definido (lenguajes con sistema de tipos estáticos), por lo tanto este mecanismo no funcionaría. 

Lenguajes como Python utilizan un sistema de tipos dinámicos, lo que permite que el tipo de una variable se determine al momento de ejecutar el código (y no al compilarlo ni al escribirlo). Gracias a esto, la función `activar` puede recibir cualquier tipo de argumentos. Sin embargo, si recibe un argumento que no posee una implementación para `gritar` o para `caminar`, se producirá un error.

## Comentarios finales

Existen muchas opiniones acerca de la relación entre polimorfismo, herencia y *duck typing* ([1](https://softwareengineering.stackexchange.com/questions/121778/is-duck-typing-a-subset-of-polymorphism), [2](https://stackoverflow.com/questions/11502433/what-is-the-difference-between-polymorphism-and-duck-typing), [3](https://www.reddit.com/r/learnprogramming/comments/2r30c0/is_ducktyping_and_advanced_form_of_polymorphism/) y otras). Lo importante para este curso es que entiendas cómo se implementan estos tres conceptos en Python. Si tienes  dudas, te invitamos a crear una issue en el foro del curso 😃.